In [1]:
#pip install gurobipy

In [2]:
from gurobipy import *
import gurobipy as gp
import pandas as pd
import numpy as np

In [3]:
c = 5

maxSpeedTruck = 15.6464
maxSpeedDrone = 22.352
SL = 40
SR = 30
E = 2400.0
M = 5313

# set of the serviceable customers for the drones
Cdrones = {1, 2, 3, 5}

# distance matrix for the selected instance
distance = [
[     0,  7663, 11389, 15022, 11557,  7801,     0],
[  7663,     0, 13324, 20798, 15531, 15388,  7663],
[ 11389, 13324,     0, 10381, 22810, 12903, 11389],
[ 15022, 20798, 10381,     0, 23513, 10161, 15022],
[ 11557, 15531, 22810, 23513,     0, 13407, 11557],
[  7801, 15388, 12903, 10161, 13407,     0,  7801],
[     0,  7663, 11389, 15022, 11557,  7801,     0]
]

# travel times matrix for vehicles
tauTruck = [
[    0.0000,  612.2015,  909.8738, 1200.1163,  923.2955,  623.2264,    0.0000],
[  612.2015,  0.0000, 1064.4621, 1661.5643, 1240.7806, 1229.3563,  612.2015],
[  909.8738, 1064.4621,    0.0000,  829.3441, 1822.3042, 1030.8282,  909.8738],
[ 1200.1163, 1661.5643,  829.3441,    0.0000, 1878.4673,  811.7682, 1200.1163],
[  923.2955, 1240.7806, 1822.3042, 1878.4673,    0.0000, 1071.0930,  923.2955],
[  623.2264, 1229.3563, 1030.8282,  811.7682, 1071.0930,    0.0000,  623.2264],
[    0.0000,  612.2015,  909.8738, 1200.1163,  923.2955,  623.2264,    0.0000]
]

# travel times matrix for drones
tauDrone = [
[    0.0000,  428.5411,  636.9117,  840.0814,  646.3068,  436.2585,    0.0000],
[  428.5411,    0.0000,  745.1235, 1163.0950,  868.5464,  860.5494,  428.5411],
[  636.9117,  745.1235,    0.0000,  580.5409, 1275.6129,  721.5797,  636.9117],
[  840.0814, 1163.0950,  580.5409,    0.0000, 1314.9271,  568.2377,  840.0814],
[  646.3068,  868.5464, 1275.6129, 1314.9271,    0.0000,  749.7651,  646.3068],
[  436.2585,  860.5494,  721.5797,  568.2377,  749.7651,    0.0000,  436.2585],
[    0.0000,  428.5411,  636.9117,  840.0814,  646.3068,  436.2585,    0.0000]
]



In [4]:
# F = set()
# for i in range(c+1):
#     for j in Cdrones:
#         if j != i:
#             for k in range(1, c+2):
#                 if k != j and k != i and tauDrone[i][j] + tauDrone[j][k] <= E:
#                     F.add((i,j,k))

# print(F)

In [5]:
#set of all the drone's tuples
F = {
   (0,1,2), (0,1,4), (0,1,5), (0,1,6), (0,2,1), (0,2,3), (0,2,5), (0,2,6), (0,3,2), (0,3,5), (0,5,1), (0,5,2), (0,5,3), (0,5,4),(0,5,6), (1,2,3), (1,2,6), (1,5,3),
(1,5,6), (2,1,6), (2,3,5), (2,3,6), (2,5,3), (2,5,6), (3,2,1), (3,2,5), (3,2,6), (3,5,1), (3,5,2), (3,5,4), (3,5,6), (4,1,6), (4,5,3), (4,5,6), (5,1,6), (5,2,3), (5,2,6),
(5,3,2), (5,3,6)
}
    

In [6]:
m = Model("FSTSP")

Set parameter Username
Academic license - for non-commercial use only - expires 2025-06-07


In [7]:
# Decision variables
x = {}
for i in range(c+1):
    for j in range(1,c+2):
        if j != i:
            x[i,j] = m.addVar(vtype = GRB.BINARY, name = f'x{i,j}')

In [8]:
y = {}
for i in range(c+1):
    for j in range(1, c+1):
        if j != i:
            for k in range(1, c+2):
                if (i,j,k) in F:
                    y[i,j,k] = m.addVar(vtype = GRB.BINARY, name = f'y{i,j,k}')

In [9]:
t_T = {}
for i in range(c+2):
    t_T[i] = m.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = f't_T{j}')
t_D = {}
for i in range(c+2):
    t_D[i] = m.addVar(vtype = GRB.CONTINUOUS, lb = 0, name = f't_D{j}')

In [10]:
p = {}
for i in range(c+1):
    for j in range(1,c+1):
        if j!= i:
            p[i,j] = m.addVar(vtype = GRB.BINARY, name = f'p{i,j}')

In [11]:
u = {}
for i in range(1, c+2):
    u[i] = m.addVar(lb = 1, ub = c+2, vtype = GRB.INTEGER, name = f'u{i}')

In [12]:
# Objective
m.setObjective(t_T[6], GRB.MINIMIZE)

In [13]:
# Constraints
#for i in range(c+1):
#       for j in range(i+1,c+1):
#             if i != j:
for j in range(1,c+1):
    m.addConstr(quicksum(x[i,j] for i in range(c+1) if i != j) + quicksum(y[i,j,k] for i in range(c+1) if i != j for k in range(1,c+2) if (i,j,k) in F) == 1)

In [14]:
m.addConstr(quicksum(x[0,j] for j in range(1,c+2)) == 1)
m.addConstr(quicksum(x[i,c+1] for i in range(c+1)) == 1)

<gurobi.Constr *Awaiting Model Update*>

In [15]:
for i in range(1,c+1):
    for j in range(1,c+2):
        if j != i:
            m.addConstr(u[i]-u[j] + 1 <= (c+2)*(1-x[i,j]))

In [16]:
for j in range(1,c+1):
    m.addConstr(quicksum(x[i,j] for i in range(c+1) if i != j) == quicksum(x[j,k] for k in range(1,c+2) if k != j))

In [17]:
for i in range(c+1):
    m.addConstr(quicksum(y[i,j,k] for j in range(1,c+1) if j != i for k in range(1,c+2) if (i,j,k) in F) <= 1)

In [18]:
for k in range(1,c+2):
    m.addConstr(quicksum(y[i,j,k] for i in range(c+1) if i != k for j in range(1,c+1) if (i,j,k) in F) <= 1)

In [19]:
for i in range(1,c+1):
    for j in range(1,c+1):
        if j != i:
            for k in range(1, c+2):
                if (i,j,k) in F:
                    m.addConstr(2*y[i,j,k] <= quicksum(x[h,i] for h in range(c+1) if h != i) + quicksum(x[l,k] for l in range(1,c+1) if l != k))

In [20]:
for j in range(1,c+1):
    for k in range(1,c+2):
        if (0,j,k) in F:
            m.addConstr(y[0,j,k] <= quicksum(x[h,k] for h in range(c+1) if h != k))

In [21]:
for i in range(1,c+1):
    for k in range(1,c+2):
        if k != i:
            m.addConstr(u[k]-u[i] >= 1 - (c+2)*(1-quicksum(y[i,j,k] for j in range(1,c+1) if (i,j,k) in F)))

In [22]:
for i in range(1,c+1):
    m.addConstr(t_D[i] >= t_T[i] - M*(1-quicksum(y[i,j,k] for j in range(1,c+1) if j != i for k in range(1,c+2) if (i,j,k) in F)))

In [23]:
for i in range(1,c+1):
    m.addConstr(t_D[i] <= t_T[i] + M*(1-quicksum(y[i,j,k] for j in range(1,c+1) if j != i for k in range(1,c+2) if (i,j,k) in F)))

In [24]:
for k in range(c+2):
    m.addConstr(t_D[k] >= t_T[k] - M*(1-quicksum(y[i,j,k] for i in range(c+1) if i != k for j in range(1,c+1) if (i,j,k) in F)))

In [25]:
for k in range(c+2):
    m.addConstr(t_D[k] <= t_T[k] + M*(1-quicksum(y[i,j,k] for i in range(c+1) if i != k for j in range(1,c+1) if (i,j,k) in F)))

In [26]:
for h in range(c+1):
    for k in range(1,c+2):
        if k != h:
            m.addConstr(t_T[k] >= t_T[h] + tauTruck[h][k] + SL*(quicksum(y[k,l,m] for l in range(1,c+1) if l!=k for m in range(1,c+2) if (k,l,m) in F)) + SR*(quicksum(y[i,j,k] for i in range(c+1) if i != k for j in range(1,c+1) if (i,j,k) in F)) - M*(1-x[h,k]))

In [27]:
for j in Cdrones:
    for i in range(c+1):
        if i != j:
            m.addConstr(t_D[j] >= t_D[i] + tauDrone[i][j]- M*(1 - quicksum(y[i,j,k] for k in range(1,c+2) if (i,j,k) in F)))

In [28]:
for j in Cdrones:
    for k in range(1,c+2):
        if k != j:
            m.addConstr(t_D[k] >= t_D[j] + tauDrone[j][k] + SR - M*(1 - quicksum(y[i,j,k] for i in range(c+1) if (i,j,k) in F)))

In [29]:
for k in range(1,c+2):
    for j in range(1,c+1):
        if j != k:
            for i in range(c+1):
                if (i,j,k) in F:
                    m.addConstr(t_D[k] - (t_D[j] - tauDrone[i][j]) <= E + M*(1-y[i,j,k]))

In [30]:
for i in range(1,c+1):
    for j in range(1,c+1):
        if j!= i:
            m.addConstr(u[i] - u[j] >= 1-(c+2)*p[i,j])

In [31]:
for i in range(1,c+1):
    for j in range(1,c+1):
        if j!= i:
            m.addConstr(u[i] - u[j] <= -1+(c+2)*(1-p[i,j]))

In [32]:
for i in range(1,c+1):
    for j in range(1,c+1):
        if j!=i:
            m.addConstr(p[i,j] + p[j,i] == 1)

In [33]:
for i in range(c+1):
    for k in range(1,c+2):
        if k != i:
            for l in range(1,c+1):
                if l != i and l != k:
                    m.addConstr(t_D[l] >= t_D[k] - M*(3-quicksum(y[i,j,k] for j in range(1,c+1) if (i,j,k) in F if j != l) - quicksum(y[l,m,n] for m in range(1,c+1) if m != i and m != k and m != l for n in range(1,c+2) if n != i and n != k and (l,m,n) in F) - p[i,l]))

In [34]:
m.addConstr(t_T[0] == 0)

<gurobi.Constr *Awaiting Model Update*>

In [35]:
m.addConstr(t_D[0] == 0)

<gurobi.Constr *Awaiting Model Update*>

In [36]:
for j in range(1,c+1):
    m.addConstr(p[0,j] == 1)

In [37]:
m.optimize()

Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 10.0 (19045.2))

CPU model: AMD E1-1200 APU with Radeon(tm) HD Graphics, instruction set [SSE2]
Thread count: 2 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 419 rows, 115 columns and 2352 nonzeros
Model fingerprint: 0x8d519e34
Variable types: 14 continuous, 101 integer (95 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e+00, 2e+04]
Presolve removed 66 rows and 18 columns
Presolve time: 0.14s
Presolved: 353 rows, 97 columns, 2121 nonzeros
Variable types: 12 continuous, 85 integer (80 binary)
Found heuristic solution: objective 5312.1644000
Found heuristic solution: objective 5287.9986000
Found heuristic solution: objective 4771.8803000

Root relaxation: objective 0.000000e+00, 39 iterations, 0.02 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds

In [38]:
m.write("FSTSP.lp")

In [39]:
if m.status == GRB.OPTIMAL:
    print("Optimal solution found")
    
    print(f"OPtimal Objective Value: {m.objVal}")
    
    for i in range(c+1):
        for j in range(1,c+2):
            if i != j and x[i,j].X > 0.5:
                print(f"x[{i},{j}] = {x[i,j].X}")
                
    for i in range(c+1):
        for j in range(1,c+1):
            if j != i:
                for k in range(1,c+2):
                    if (i,j,k) in F:
                        if y[i,j,k].X > 0.5:
                            print(f"y[{i},{j},{k}] = {y[i,j,k].X}")
                            
    for i in range(1, c+2):
        print(f"u[{i}] = {u[i].X}")
        
    for i in range(1,c+2):
        if t_T[i].X > 0.5:
            print(f"t_T[{i}] = {t_T[i].X}") 
    
    for i in range(1,c+2):
        if t_D[i].X > 0.5:
            print(f"t_D[{i}] = {t_D[i].X}")
    
    for i in range(c+1):
        for j in range(1,c+1):
            if j != i and p[i,j].X > 0.5: 
                print(f"p[{i},{j}] = {p[i,j].X}")
                
    

Optimal solution found
OPtimal Objective Value: 3935.9462999998914
x[0,1] = 1.0
x[1,5] = 1.0
x[4,6] = 1.0
x[5,4] = 1.0
y[0,2,5] = 1.0
y[5,3,6] = 1.0
u[1] = 2.0
u[2] = 3.0
u[3] = 1.0
u[4] = 6.0
u[5] = 4.0
u[6] = 7.0
t_T[1] = 612.2014999999856
t_T[2] = 615.915299522675
t_T[3] = 384.05209952256473
t_T[4] = 2982.6507999999117
t_T[5] = 1911.5577999999412
t_T[6] = 3935.9462999998914
t_D[2] = 1159.978099999893
t_D[3] = 3065.864899999812
t_D[5] = 1911.5577999998932
t_D[6] = 3935.9462999998123
p[0,1] = 1.0
p[0,2] = 1.0
p[0,3] = 1.0
p[0,4] = 1.0
p[0,5] = 1.0
p[1,2] = 1.0
p[1,4] = 1.0
p[1,5] = 1.0
p[2,4] = 1.0
p[2,5] = 1.0
p[3,1] = 1.0
p[3,2] = 1.0
p[3,4] = 1.0
p[3,5] = 1.0
p[5,4] = 1.0
